In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, f1_score

2023-11-12 17:20:29.373370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 17:20:30.069574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-11-12 17:20:30.069665: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
# Data directories
train_data_dir = '../Data/images/train'
test_data_dir = '../Data/images/test'


In [3]:
# Data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [4]:
# Load data
batch_size = 128
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 21077 images belonging to 4 classes.
Found 5140 images belonging to 4 classes.


In [5]:
# Load the ResNet-101 model
base_model = ResNet101(weights='imagenet', include_top=False)

# Add a custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

2023-11-12 17:20:31.608384: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 17:20:31.632160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 17:20:31.632401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 17:20:31.632932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Train the model
epochs = 25
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

Epoch 1/25


2023-11-12 17:20:39.297284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-11-12 17:20:39.594680: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-12 17:20:40.460705: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.56GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 17:20:40.460740: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.56GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 17:20:42.100167: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate

133/164 [=======================>......] - ETA: 24s - loss: 1.4162 - accuracy: 0.3019

2023-11-12 17:22:28.296358: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.36GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 17:22:28.296400: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.36GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 17:22:28.375194: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.64GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 17:22:28.375226: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU

164/164 [==============================] - 167s 978ms/step - loss: 1.4147 - accuracy: 0.3044 - val_loss: 1.3653 - val_accuracy: 0.3324
Epoch 2/25
164/164 [==============================] - 155s 939ms/step - loss: 1.3519 - accuracy: 0.3470 - val_loss: 1.3216 - val_accuracy: 0.4047
Epoch 3/25
164/164 [==============================] - 156s 946ms/step - loss: 1.3468 - accuracy: 0.3590 - val_loss: 1.3284 - val_accuracy: 0.3781
Epoch 4/25
164/164 [==============================] - 155s 944ms/step - loss: 1.3395 - accuracy: 0.3588 - val_loss: 1.3147 - val_accuracy: 0.3721
Epoch 5/25
164/164 [==============================] - 154s 938ms/step - loss: 1.3337 - accuracy: 0.3665 - val_loss: 1.3008 - val_accuracy: 0.3957
Epoch 6/25
164/164 [==============================] - 160s 975ms/step - loss: 1.3230 - accuracy: 0.3713 - val_loss: 1.3048 - val_accuracy: 0.3781
Epoch 7/25
164/164 [==============================] - 167s 1s/step - loss: 1.3243 - accuracy: 0.3713 - val_loss: 1.3015 - val_accuracy:

In [8]:
# Evaluation
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = tf.argmax(y_pred, axis=1)

f1 = f1_score(y_true, y_pred_classes, average='weighted')
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

print(f'F1 Score: {f1}')
print(report)


41/41 [==============================] - 29s 705ms/step
F1 Score: 0.25900311868044296
              precision    recall  f1-score   support

       angry       0.32      0.08      0.12       960
       happy       0.38      0.96      0.54      1825
     neutral       0.46      0.03      0.06      1216
         sad       0.48      0.08      0.13      1139

    accuracy                           0.38      5140
   macro avg       0.41      0.29      0.21      5140
weighted avg       0.41      0.38      0.26      5140

